# rdsad

Process .rds files to .h5ad files.

In [ ]:
import rpy2
import rpy2.robjects as robjects


In [ ]:
seurat = robjects.r("library(Seurat)")
seurat_data = robjects.r('library(SeuratData)')
seurat_disk = robjects.r('library(SeuratDisk)')
fastmat = robjects.r('library(fastMatMR)')

In [ ]:
read_rds = robjects.r['readRDS']
rds_data = read_rds('data/JK06.rds')

In [ ]:
create_seurat_object = robjects.r['CreateSeuratObject']

In [ ]:
seurat_object = create_seurat_object(rds_data)

In [ ]:
get_assay_data = robjects.r['GetAssayData']

In [ ]:
mtx = get_assay_data(seurat_object, assay = "RNA", layer = "counts")

In [ ]:
write_matrix = robjects.r['write_fmm']

In [ ]:
write_matrix(mtx, "data/matrix.mtx")

In [ ]:
rownames = robjects.r["rownames"]

In [ ]:
barcodes = rownames(seurat_object.slots["meta.data"])
# print(f"{barcodes}")

In [ ]:
robjects.r["str"](seurat_object)

In [ ]:
# 
# need to extract the features from the v5 Assay object buried inside the S4 object.
#
x = tuple(seurat_object.slots["assays"].items())[0][1]
genes = rownames(x.slots["features"])

In [ ]:
write_table = robjects.r["write.table"]


write_table(barcodes, file="data/barcodes.tsv", quote=False, row_names=False, col_names=False)
write_table(genes, file="data/genes.tsv", quote=False, row_names=False, col_names=False)

In [ ]:
from anndata import AnnData
import scipy.io
import pandas as pd

In [ ]:
mtx = scipy.io.mmread("data/matrix.mtx").tocsr().T # need to check if this is the correct sparse matrix format

In [ ]:
adata = AnnData(mtx)
adata.X

In [ ]:
barcodes = pd.read_csv("data/barcodes.tsv", header=None)
features = pd.read_csv("data/genes.tsv", header=None)

In [ ]:
adata.var = features
adata.var_names = adata.var[0]

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
adata.obs.drop(adata.obs[[0]], axis=1, inplace=True)

In [ ]:
adata.obs